In [3]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pandas.plotting import table
from matplotlib.table import table
from bs4 import BeautifulSoup
import base64
from io import BytesIO

file = "/workspaces/SPL/Middleput/points.xlsx"
season1_excel = "/workspaces/SPL/Middleput/season1.xlsx"
season2_excel = "/workspaces/SPL/Middleput/season2.xlsx"
points = pd.read_excel(file)
season1_sorted = pd.read_excel(season1_excel)
season2_sorted = pd.read_excel(season2_excel)
def mpl_to_html(fig, css_link="/Middleput/player_data/styles_graphs.css"):
    """
    Convert a Matplotlib Figure object into a Base64-encoded PNG and return an HTML string.
    """
    buf = BytesIO()
    fig.savefig(buf, format="png")
    buf.seek(0)
    image_base64 = base64.b64encode(buf.read()).decode('utf-8').replace('\n', '')
    buf.close()
    
    # Enhanced HTML structure
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Player Graph</title>
        <link rel="stylesheet" href="{css_link}">
    </head>
    <body>
        <div class="graph-container">
            <img src='data:image/png;base64,{image_base64}' alt="Player Graph">
        </div>
    </body>
    </html>
    """
    
    return html_content

def player_graphs_with_summary_updated(season_points, season, player_name, sorted_1, sorted_2):
    player_data_season = season_points[season_points['Season'] == season]
    player_data = player_data_season[player_data_season['Player'] == player_name]
    
    if season == 1:
        season_sorted = sorted_1
    else:   
        season_sorted = sorted_2

    player_summary = season_sorted[season_sorted['Player'] == player_name]
    
    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return
    
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), gridspec_kw={"height_ratios": [0.2, 0.4, 0.4]})
    axes[0, 0].axis("off")
    axes[0, 1].axis("off")
    ax_center = fig.add_subplot(3, 2, (1, 2))
    ax_center.axis("off")
    
    player_summary = player_summary.drop(columns=['Player']).T
    player_summary.columns = [player_name]
    cell_text = player_summary.reset_index().values.tolist()
    tbl = table(ax_center, cellText=cell_text, loc="center", cellLoc="center")
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(10)
    tbl.scale(1.0, 1.2)
    
    sns.lineplot(x='Gameweek', y='Total Points', data=player_data, ax=axes[1, 0], marker="o")
    sns.barplot(x='Gameweek', y='Goal Points', data=player_data, ax=axes[1, 1], palette="viridis")
    game_outcomes = player_data['Game Outcome'].value_counts()
    axes[2, 0].pie(game_outcomes, labels=game_outcomes.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"))
    sns.lineplot(x='Gameweek', y='Defensive Score Points', data=player_data, ax=axes[2, 1], label='Defensive Score', marker="o")
    sns.lineplot(x='Gameweek', y='Midfield Score', data=player_data, ax=axes[2, 1], label='Midfield Score', marker="o")
    
    plt.tight_layout()
    return fig

def save_player_graphs_to_html(season_points, season, sorted_1, sorted_2):
    directory_path = f"/workspaces/SPL/Middleput/player_data/s{season}/player_graphs"
    
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    
    points = season_points[season_points['Season'] == season]
    players = points['Player'].unique()

    for player in players:
        fig = player_graphs_with_summary_updated(points, season, player, sorted_1, sorted_2)
        
        if fig:  # Only save if a figure is returned (i.e., player data exists)
            html_content = mpl_to_html(fig)
            with open(f"{directory_path}/{player}.html", "w") as file:
                file.write(html_content)
            plt.close(fig)

    return f"Graphs saved for {len(players)} players in {directory_path}"

#save_player_graphs_to_html(points, 1, season1_sorted, season2_sorted)
save_player_graphs_to_html(points, 2, season1_sorted, season2_sorted)


In [7]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import os

# Read the data
file = "/workspaces/SPL/Middleput/points.xlsx"
season1_excel = "/workspaces/SPL/Middleput/season1.xlsx"
season2_excel = "/workspaces/SPL/Middleput/season2.xlsx"
points = pd.read_excel(file)
season1_sorted = pd.read_excel(season1_excel)
season2_sorted = pd.read_excel(season2_excel)

def player_graphs_with_summary_updated(season_points, season, player_name, sorted_1, sorted_2):
    player_data_season = season_points[season_points['Season'] == season]
    player_data = player_data_season[player_data_season['Player'] == player_name]
    
    if season == 1:
        season_sorted = sorted_1
    else:   
        season_sorted = sorted_2

    player_summary = season_sorted[season_sorted['Player'] == player_name]
    
    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return None

    # Creating the subplots using plotly
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        subplot_titles=('Total Points vs Gameweek', 'Goal Points vs Gameweek', 'Game Outcomes', 'Defensive and Midfield Scores vs Gameweek'),
        vertical_spacing=0.1,
        specs=[[{}], [{}], [{'type': 'domain'}], [{}]]
    )

    # Line plot for Total Points vs Gameweek
    fig.add_trace(go.Scatter(x=player_data['Gameweek'], y=player_data['Total Points'], mode='lines+markers', name='Total Points'), row=1, col=1)
    
    # Bar plot for Goal Points vs Gameweek
    fig.add_trace(go.Bar(x=player_data['Gameweek'], y=player_data['Goal Points'], name='Goal Points'), row=2, col=1)

    # Pie chart for Game Outcomes
    game_outcomes = player_data['Game Outcome'].value_counts()
    fig.add_trace(go.Pie(labels=game_outcomes.index, values=game_outcomes, hole=.3), row=3, col=1)

    # Line plots for Defensive and Midfield Scores vs Gameweek
    fig.add_trace(go.Scatter(x=player_data['Gameweek'], y=player_data['Defensive Score Points'], mode='lines+markers', name='Defensive Score'), row=4, col=1)
    fig.add_trace(go.Scatter(x=player_data['Gameweek'], y=player_data['Midfield Score'], mode='lines+markers', name='Midfield Score'), row=4, col=1)

    # Styling
    fig.update_layout(
        title=f"Performance of {player_name} in Season {season}",
        hovermode="x"
    )
    fig.update_xaxes(title_text="Gameweek")
    fig.update_yaxes(title_text="Points", row=1, col=1)
    fig.update_yaxes(title_text="Points", row=2, col=1)
    fig.update_yaxes(title_text="Percentage", row=3, col=1)
    fig.update_yaxes(title_text="Points", row=4, col=1)

    return fig

# Test
save_player_graphs_to_html(points, 2, season1_sorted, season2_sorted)


'Graphs saved for 9 players in /workspaces/SPL/Middleput/player_data/s2/player_graphs'